In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime

In [2]:
subscription_schema = pa.schema([
    ('timestamp', pa.timestamp('ms')),
    ('id', pa.int32()),
    ('email', pa.string())
])

In [8]:
timestamps = pa.array([
    datetime(2019, 9, 3, 9, 0, 0),
    datetime(2019, 9, 3, 10, 0, 0),
    datetime(2019, 9, 3, 11, 0, 0)
], type = pa.timestamp('ms'))

ids = pa.array([1, 2, 3], type = pa.int32())

emails = pa.array(
    ['first@example.com', 'second@example.com', 'third@example.com'],
    type = pa.string()
)

batch = pa.RecordBatch.from_arrays(
    [timestamps, ids, emails],
    names = subscription_schema
)

batch

C:\Users\alibh\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Schema passed to names= option, please pass schema= explicitly. Will raise exception in future
  app.launch_new_instance()


In [12]:
table = pa.Table.from_batches([batch])
pq.write_table(table, 'subscriptions.parquet')


In [15]:
dataframe = pd.DataFrame([
    [datetime(2019, 9, 3, 9, 0, 0), 1, 'first@example.com'],
    [datetime(2019, 9, 3, 10, 0, 0), 1, 'second@example.com'],
    [datetime(2019, 9, 3, 11, 0, 0), 1, 'third@example.com'],
], columns = ['timestamp', 'id', 'email'])

dataframe
table_from_pandas = pa.Table.from_pandas(dataframe)
pq.write_table(table_from_pandas, 'subscriptions_pandas.parquet')


In [17]:
pq.write_to_dataset(
    table,
    root_path='subscriptions_partitioned.parquet',
    partition_cols=['id']
)